In [80]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from IPython.display import clear_output

In [81]:
# 요청 함수
def getSource(site) :
    # 해더 정보 셋팅
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    # 요청한다.
    response = requests.get(site, headers=header_info)
    return response.text

In [87]:
# 한 페이지의 데이터를 가져오는 함수
def getData(source):
    # bs4 객체를 생성한다.
    soup = bs4.BeautifulSoup(source)
    
    # li 태그들을 가져온다.
    li_list = soup.select('body > div > div > div.score_result > ul > li')
    
    data_dict = {
        '평점' : [],
        '작성자' : [],
        '작성시간' : [],
        '공감수' : [],
        '비공감수' : [],
        '평가글' : [],
    }
    
    for idx, li_tag in enumerate(li_list):
        # print(li_tag)
        # print('-------------------------')
        
        # 평점
        a1 = li_tag.select_one('div.star_score > em')
        data1 = getText(a1)
        
        # 작성자
        a2 = li_tag.select_one('a > span')
        data2 = getText(a2)
        
        # 작성시간
        a3 = li_tag.select_one('div.score_reple > dl > dt > em:nth-child(2)')
        data3 = getText(a3)
        
        # 공감수
        a4 = li_tag.select_one('div.btn_area > a._sympathyButton > strong')
        data4 = getText(a4)
        
        # 비공감수
        a5 = li_tag.select_one('div.btn_area > a._notSympathyButton > strong')
        data5 = getText(a5)
        
        # 평가글
        # 접혀져 있는 글을 가져온다.
        a6 = li_tag.select_one(f'#_unfold_ment{idx} > a')
        if a6 != None :
            a7 = a6.attrs['data-src']
            data6 = a7.strip()
        else :
            a7 = li_tag.select_one(f'#_filtered_ment_{idx}')
            data6 = getText(a7)
        
        # 데이터를 담는다.
        data_dict['평점'].append(data1)
        data_dict['작성자'].append(data2)
        data_dict['작성시간'].append(data3)
        data_dict['공감수'].append(data4)
        data_dict['비공감수'].append(data5)
        data_dict['평가글'].append(data6)
        
    df1 = pd.DataFrame(data_dict)
    
    if os.path.exists('./05_data.csv') == False :
        df1.to_csv('./05_data.csv', encoding='utf-8-sig', index=False)
    else :
        df1.to_csv('./05_data.csv', encoding='utf-8-sig', index=False, header=None, mode='a')

        # print(data1)
        # print(data2)
        # print(data3)
        # print(data4)
        # print(data5)
        # print(data6)
        # print('-------------------------')

In [88]:
# 전달된 태그로 부터 문자열 데이터를 추출해 반환하는 함수
def getText(tag):
    # 태그가 없다면
    if tag == None :
        return np.nan
    # 문자열을 가져온다.
    data = tag.text.strip()
    # 길이가 0 이라면...
    if len(data) == 0 :
        return np.nan
    return data

In [89]:
def getPageCnt(source) :
    soup = bs4.BeautifulSoup(source)
    
    # 전체 글 개수를 가져온다.
    a1 = soup.select_one('body > div > div > div.score_total > strong > em')
    data1 = getText(a1)
    # print(data1)
    
    # 쉼표를 제거한다.
    data1 = data1.replace(',', '')
    # 공백 제거한다.
    data1 = data1.strip()
    # 숫자로 변환한다.
    data2 = int(data1)
    # 10으로 나눈 몫을 구한다.
    page_cnt = data2 // 10
    # 10으로 나눈 나머지를 구해 0이 아니면 1을 증가한다.
    if data2 % 10 > 0 :
        page_cnt = page_cnt + 1
    return page_cnt

In [90]:
# 데이터를 불러온다.
df1 = pd.read_csv('./04_data.csv')

# 제목과 영화코드만 추출한다.
df2 = df1[['제목', '영화코드']]

# 영화의 수만큼 반복한다.
movie_cnt = len(df2)

for idx in range(movie_cnt):
    a1 = df2.iloc[idx]
    title = a1['제목']
    code = a1['영화코드']
    
    # print(title)
    # print(code)
    # print('------------------')
    
    site = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'
    time.sleep(1)
    clear_output(wait=True)
    
    # 요청한다.
    source = getSource(site)
    # print(source)
    page_cnt = getPageCnt(source)
    
    # 페이지 수 만큼 반복한다.
    for page in range(page_cnt):
        time.sleep(1)
        clear_output(wait=True)
        
        print(f'code : {code}, {page + 1} / {page_cnt}')
        
        site = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page+1}'
        # 현재 페이지를 요청한다.
        source2 = getSource(site)
        # 현재 페이지의 글정보를 가져온다.
        getData(source2)

code : 214314, 1 / 7


KeyboardInterrupt: 